In [ ]:
#TEST

In [ ]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
import matplotlib
import matplotlib.pyplot as plt
import scipy.optimize as optimize
import Estimate
import figure 
from EconModel import cpptools
import pickle

from LimitedCommitmentModel import LimitedCommitmentModelClass

# plot style
linestyles = ['-','--','-.',':',':']
markers = ['o','s','D','*','P']
linewidth = 2
font_size = 17
font = {'size':font_size}
matplotlib.rc('font', **font)


plt.rcParams.update({'figure.max_open_warning': 0,'text.usetex': False})
path = 'output/'

SAVE = False

# c++ settings

do_compile = True
threads = 20
T = 20


In [ ]:
name = 'model_NC'
model_name = "model_NC"
do_distr_yerror = "none"
do_yerror = 0
name_error = f'{do_distr_yerror}{do_yerror}std_Yerror'
#N = 16000 #MODEL_LC 
#N = 4000 #model NC
N = 3000
#name_error = "uni2std_Yerror"

specs = {}
#specs.update({f'test2':{'latexname':'limited', 'par':{ 'T':T, 'num_K': 5, 'num_A': 10 ,  'do_HK': False, 'threads':threads,'bargaining':0}}})


#specs.update({f'test2':{'latexname':'limited', 'par':{ 'T':T, 'do_HK': False, 'simN': N,  'threads':threads,'bargaining':1}}})

#specs.update({f'model_LC_highsigmaHK':{'latexname':'limited', 'par':{ 'T':T, 'do_HK': False, 'simN': N,  'threads':threads,'bargaining':1}}})
#specs.update({f'model_LC':{'latexname':'limited', 'par':{ 'T':T, 'do_HK': False, 'simN': N,  'threads':threads,'bargaining':1}}})specs.update({f'model_NC':{'latexname':'limited', 'par':{ 'T':T, 'do_HK': False, 'simN': N,  'threads':threads,'bargaining':2}}})
#specs.update({f'model_FC':{'latexname':'limited', 'par':{ 'T':T,  'do_HK': False, 'simN': N,  'threads':threads,'bargaining':0}}})

# solve different models
models = {}
for m,(name,spec) in enumerate(specs.items()):
    
    # setup model
    model = LimitedCommitmentModelClass(name=name,par=spec['par'])
    model.spec = spec

    compile_now = True if do_compile & (m==0) else False
    model.link_to_cpp(force_compile=compile_now)
    

# setup model
#model = LimitedCommitmentModelClass(name=name)    
#model.link_to_cpp(force_compile=True)


#load par and sol
pickle_file = f'output/{name}_sol.pickle'

with open(pickle_file, 'rb') as f:
    model.sol = pickle.load(f)

pickle_file = f'output/{name}_par.pickle'

with open(pickle_file, 'rb') as f:
    model.par = pickle.load(f)
    model.par.simN = N

In [ ]:
#model.par.sigma_K = 1.0 # men jeg har løst det med det forkerte.... 
#model.par.sigma_love =  0.1 

data = Estimate.create_data(model,start_p = 2, end_p = model.par.T-2, yerror = do_distr_yerror, scale_st = do_yerror )
data = Estimate.create_variable(data, model.par)
#data = Estimate.aux_est(data,model.par)

data2 = data

#data2 = data2[data2['delta_log_barganing']< 0.5]
#data2 = data2[data2['delta_log_barganing']> -0.5]
#data2 = data2[data2['delta_log_barganing']!= 0]
print(data[['idx', 't','barganing','hours_w','hours_m','wealth']])

_, wald_FC_w , wald_NC_w,  _,_,_ = Estimate.main_est(data2,gender= "w", do_estimate_wage = "est_omega", print_reg = True, shadow_value_simple=4, do_control_love = True, part_earning_simple =3, control_cons = 3, wealth_love = 1, do_true_barg = True)
   

In [ ]:
#SUMMARIZE!! TORSDAG!!  N = 5000, FC: 5066 NC: 19,000 , LC: 21,000
data2 = Estimate.create_data(model,start_p = 2, end_p = model.par.T-2, yerror = do_distr_yerror, scale_st = do_yerror )
data2 = Estimate.create_variable(data2, model.par)
data2['BMI_w']=data2['BMI_w']-1
data2['BMI_m']=data2['BMI_m']-1
summary = data2[['hours_w','hours_m','cons','wage_w','wage_m','earnings_w','earnings_m','wealth','BMI_w','BMI_m','barganing']].describe()
summary_by_T = data2[['hours_w','hours_m','cons','wage_w','wage_m','earnings_w','earnings_m','wealth','BMI_w','BMI_m','barganing','t']].groupby('t').mean()
summary_by_T['div_share'] = 1-data2[['hours_w','t']].groupby('t').count()/N

print(summary)
print(summary_by_T)

summary.to_excel(f'summary_statistics_{name}.xlsx')
summary_by_T.to_excel(f'summary_statistics_byT_{name}.xlsx')

#print(summary_by_count)

In [ ]:
#kig på ændring i hours

gender = "w"

data2['rel_dif_hours_w']=data2['delta_hours_w']/data2['hours_w_l']

#kig på andel af par hvor en arbejder fuldtid


#stor en adnel er ved 1 

data2['hours_w_limit100'] = (data2['hours_w'] == 1)
data2['hours_w_limit100'] = data2['hours_w_limit100'].replace({True: 1, False: 0})
data2['hours_w_limit97'] = (data2['hours_w'] > 0.97)
data2['hours_w_limit97'] = data2['hours_w_limit97'].replace({True: 1, False: 0})

data2['hours_w_limit90'] = (data2['hours_w'] > 0.90)
data2['hours_w_limit90'] = data2['hours_w_limit90'].replace({True: 1, False: 0})

data2['hours_m_limit'] = (data2['hours_m'] > 0.97)
data2['hours_m_limit'] = data2['hours_m_limit'].replace({True: 1, False: 0})


print(data2[['hours_w_limit97','hours_w','hours_m_limit','barganing']])



summary = data2[['rel_dif_hours_w','delta_hours_w','hours_w_l', 'hours_w_limit100', 'hours_w_limit97', 'hours_w_limit90']].describe([.01,.1,.25,.5,.75,.9,.99])
print(summary)
#antag små ændringer er + - 10%

plt.hist(data2['hours_w'], bins = 50, density = True)
plt.xlabel('Hours')
plt.ylabel('PDF')
plt.savefig(f'output/hours_plot{model_name}_{gender}.png')
plt.show()

data3 = data2
data3=data3.drop(data3[(data3['hours_w_l']>0.9999) | (data3['hours_m_l']>0.9999) | (data3['hours_w']>0.9999) | (data3['hours_m']>0.9999)].index)

plt.hist(data2['rel_dif_hours_w'], bins = 50, density = True)
plt.xlabel('Percentage change in hours')
plt.ylabel('PDF')

plt.savefig(f'output/change_in_hours_plot{model_name}_{gender}.png')
#plt.hist(data3['rel_dif_hours_w'], bins = 50)
plt.show()



plt.hist(data2['wealth'], bins = 50, density = True)
plt.xlabel('wealth')
plt.ylabel('PDF')
plt.show()


In [ ]:
#bargaining
#kig på ændring i hours

gender = "w"

data2['rel_dif_barg']=data2['delta_barganing']/data2['barganing_l']

#kig på andel af par hvor en arbejder fuldtid


#stor en adnel er ved 1 



data2['delta_bargaining0'] = (data2['delta_barganing'] == 0)
data2['delta_bargaining0'] = data2['delta_bargaining0'].replace({True: 1, False: 0})
summary = data2[['delta_bargaining0','delta_barganing','barganing_l','rel_dif_barg']].describe([.01,.1,.25,.5,.75,.9,.99])
print(summary)
#antag små ændringer er + - 10%

plt.hist(data2['barganing'], bins = 50, density = True)
plt.xlabel('Barganing')
plt.ylabel('PDF')
plt.savefig(f'output/barganing_plot{model_name}_{gender}.png')
plt.show()

data3 = data2
#data3=data3.drop(data3[data3['barganing']== 0.5].index)

plt.hist(data2['rel_dif_barg'], bins = 50, density = True)
#plt.hist(data3['rel_dif_barg'], bins = 50, density = True)
plt.xlabel('Percentage change in barganing')
plt.ylabel('PDF')

plt.savefig(f'output/change_in_barg_plot{model_name}_{gender}.png')
#plt.hist(data3['rel_dif_hours_w'], bins = 50)
plt.show()



data3 = data2
data3=data3.drop(data3[data3['rel_dif_barg']== 0.0].index)

plt.hist(data3['rel_dif_barg'], bins = 50, density = True)
#plt.hist(data3['rel_dif_barg'], bins = 50, density = True)
plt.xlabel('Percentage change in barganing')
plt.ylabel('PDF')

plt.savefig(f'output/change_in_barg_plot{model_name}_{gender}_no0.png')
#plt.hist(data3['rel_dif_hours_w'], bins = 50)
plt.show()

